In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,18699
2,Huelva,Confirmados PDIA 14 días,5382
3,Huelva,Tasa PDIA 14 días,"1053,7589355115979"
4,Huelva,Confirmados PDIA 7 días,2707
5,Huelva,Total Confirmados,18913
6,Huelva,Curados,10908
7,Huelva,Fallecidos,161


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  18699.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5034.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 233 personas en los últimos 7 días 

Un positivo PCR cada 114 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,18699.0,2707.0,5382.0,510743.0,530.012159,1053.758936,287.0
Huelva-Costa,11335.0,1819.0,3647.0,288115.0,631.345123,1265.813998,158.0
Condado-Campiña,5079.0,672.0,1176.0,155057.0,433.389012,758.430771,93.0
Huelva (capital),5034.0,617.0,1264.0,143663.0,429.477318,879.836840,89.0
Sierra de Huelva-Andévalo Central,2032.0,209.0,498.0,67571.0,309.304287,737.002560,33.0
Isla Cristina,1575.0,354.0,833.0,21264.0,1664.785553,3917.419112,16.0
Punta Umbría,454.0,137.0,190.0,15242.0,898.832174,1246.555570,15.0
Bollullos Par del Condado,366.0,52.0,89.0,14272.0,364.349776,623.598655,14.0
Palma del Condado (La),784.0,83.0,155.0,10761.0,771.303782,1440.386581,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,366.0,85.0,181.0,3338.0,2546.434991,5422.408628,12.0
Encinasola,109.0,4.0,58.0,1305.0,306.513410,4444.444444,0.0
Isla Cristina,1575.0,354.0,833.0,21264.0,1664.785553,3917.419112,16.0
Villanueva de las Cruces,16.0,1.0,14.0,373.0,268.096515,3753.351206,0.0
Berrocal,11.0,5.0,11.0,312.0,1602.564103,3525.641026,1.0
Paymogo,45.0,17.0,33.0,1159.0,1466.781708,2847.282140,7.0
Calañas,143.0,25.0,78.0,2791.0,895.736295,2794.697241,1.0
Cerro de Andévalo (El),116.0,19.0,49.0,2364.0,803.722504,2072.758037,1.0
San Bartolomé de la Torre,124.0,40.0,69.0,3736.0,1070.663812,1846.895075,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Encinasola,109.0,4.0,58.0,1305.0,306.513410,4444.444444,0.0,0.068966
Villanueva de las Cruces,16.0,1.0,14.0,373.0,268.096515,3753.351206,0.0,0.071429
Minas de Riotinto,94.0,2.0,21.0,3848.0,51.975052,545.738046,0.0,0.095238
Aroche,79.0,1.0,5.0,3073.0,32.541490,162.707452,0.0,0.200000
Villarrasa,117.0,6.0,25.0,2176.0,275.735294,1148.897059,1.0,0.240000
Zarza-Perrunal (La),19.0,1.0,4.0,1267.0,78.926598,315.706393,0.0,0.250000
Almonaster la Real,40.0,1.0,4.0,1819.0,54.975261,219.901045,0.0,0.250000
Almendro (El),32.0,2.0,8.0,826.0,242.130751,968.523002,0.0,0.250000
Calañas,143.0,25.0,78.0,2791.0,895.736295,2794.697241,1.0,0.320513
